In [1]:
import os
import numpy as np
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n
from torch_geometric.data import Data

In [2]:
def bump(g):
    return g
    # return Data.from_dict(g.__dict__)

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
      prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
      prot_1 = bump(prot_1)
      prot_2 = bump(prot_2)
      return prot_1, prot_2, torch.tensor(self.label[index])

In [3]:
npy_file = "../human_features/npy_file_new(human_dataset).npy"
npy_file_test = "../human_features/npy_file_test.npy"
processed_dir="../human_features/esm/"
# train = LabelledDataset(npy_file=npy_file, processed_dir=processed_dir)
# test = LabelledDataset(npy_file=npy_file_test, processed_dir=processed_dir)

In [4]:
# TRAINING set  
from tqdm.auto import tqdm

X_train = []
y_train = []
mapping_train = np.load(npy_file)
bad_rows = []
for row in tqdm(mapping_train):
    try:
        _, _, prot1, _, _, prot2, label = row
        prot_1 = os.path.join(processed_dir, prot1+".pt")
        prot_2 = os.path.join(processed_dir, prot2+".pt")
        prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
        prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
        p = torch.cat([prot_1, prot_2])
        label = float(label)
        X_train.append(p)
        y_train.append(label)
    except:
        bad_rows.append(row)

  0%|          | 0/97316 [00:00<?, ?it/s]

In [12]:
len(X_train)

97014

In [13]:
len(y_train)

97014

In [25]:
# TESTING set 
X_test = []
y_test = []
bad_test_rows = []
mapping_test = np.load(npy_file_test)
for row in tqdm(mapping_test):
    try:
        _, _, prot1, _, _, prot2, label = row
        prot_1 = os.path.join(processed_dir, prot1+".pt")
        prot_2 = os.path.join(processed_dir, prot2+".pt")
        prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
        prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
        p = torch.cat([prot_1, prot_2])
        label = float(label)
        X_test.append(p)
        y_test.append(label)
    except:
        bad_test_rows.append(row)

  0%|          | 0/5922 [00:00<?, ?it/s]

In [15]:
len(X_test)

5907

In [16]:
len(y_test)

5907

In [26]:
for i in range(len(X_train)):
    X_train[i] = X_train[i][None, :]

In [27]:
X_train = torch.cat(X_train)

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [28]:
!pip3 install -qq scikit-learn 

In [29]:
from sklearn.model_selection import train_test_split

train, val, ytrain, yval = train_test_split(X_train, y_train, test_size=0.2)

In [30]:
ytrain = [x for x in ytrain]
yval = [x for x in yval]

In [31]:
# TRAINING 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)

model = nn.Sequential(
    nn.Linear(in_features=X_train.shape[1], out_features=1)
)

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

dataset = TensorDataset(X_train_torch, y_train_torch)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_epochs = 10

for epoch in tqdm(range(num_epochs)):
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        logits = model(X_batch).squeeze()
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    logits = model(X_train_torch).squeeze(dim=1)
    probs = torch.sigmoid(logits)
    print("Example predictions:", probs[:10])

/tmp/ipykernel_7603/3477652198.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_torch = torch.tensor(X_train, dtype=torch.float32)


  0%|          | 0/10 [00:00<?, ?it/s]

Example predictions: tensor([0.3929, 0.3369, 0.3502, 0.4568, 0.3209, 0.2964, 0.2664, 0.2765, 0.3402,
        0.3889])
